In [1]:
import dask.array as da

In [6]:
da.random.randint(0,255, (4096 * 2,) * 3, chunks=(512,) * 3, dtype='uint8')

,Array,Chunk
Bytes,512.00 GiB,128.00 MiB
Shape,"(8192, 8192, 8192)","(512, 512, 512)"
Count,4096 Tasks,4096 Chunks
Type,uint8,numpy.ndarray


In [1]:
# Create a zarr array
import zarr
import numpy as np

data = np.arange(16, dtype='uint8')

zarray = zarr.open('test.zarr', # Path to the root of the zarr hierarchy 
                  path='foo',  # Path to the array I am creating
                  shape=data.shape,  # Array shape
                  dtype=data.dtype, # Array datatype
                  chunks=(data.size//4,), # Array chunking
                  mode='w') # Storage access mode

zarray[:] # By default, the array is full of 0s

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [2]:
!tree -a test.zarr

test.zarr
├── .zgroup
└── foo
    └── .zarray

1 directory, 2 files


In [4]:
zarray[:] = data
zarray[:]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
      dtype=uint8)

In [5]:
!tree -a test.zarr

test.zarr
├── .zgroup
└── foo
    ├── .zarray
    ├── 0
    ├── 1
    ├── 2
    └── 3

1 directory, 6 files


In [6]:
zarray.attrs['property'] = {'numbers': [0,1,2]}

In [7]:
dict(zarray.attrs)

{'property': {'numbers': [0, 1, 2]}}

In [8]:
!cat test.zarr/foo/.zattrs

{
    "property": {
        "numbers": [
            0,
            1,
            2
        ]
    }
}

In [10]:
zarray.info

Name,/foo
Type,zarr.core.Array
Data type,uint8
Shape,"(16,)"
Chunk shape,"(4,)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,16
No. bytes stored,492


In [11]:
import dask.array as da
darray = da.arange(16,32, chunks=(4,), dtype='uint8')

In [12]:
storage_op = darray.to_zarr('test.zarr', 
                            component='foo', 
                            overwrite=True, 
                            compute=False)
storage_op.compute()

[None, None, None, None]

In [13]:
zarray[:]

array([16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
      dtype=uint8)

In [15]:
storage_op = darray.store(zarray, 
                          compute=False, 
                          lock=None)
storage_op.compute()

[None, None, None, None]

In [16]:
import dask
def store_block(source, target, region, block_info=None):
    chunk_origin = block_info[0]["array-location"]
    slices = tuple(slice(start, stop) for start, stop in chunk_origin)
    if region:
        slices = da.optimization.fuse_slice(region, slices)
    target[slices] = source
    return np.expand_dims(0, tuple(range(source.ndim)))
  
    
def store_blocks(sources, targets, regions=None):
    result = []

    if isinstance(sources, dask.array.core.Array):
        sources = [sources]
        targets = [targets]

    if isinstance(regions, tuple) or regions is None:
        regions = [regions]
    for source, target, region in zip(sources, targets, regions):
        out_chunks = tuple((1,) * len(c) for c in source.chunks)
        result.append(
            da.map_blocks(
                store_block,
                source,
                target,
                region,
                chunks=out_chunks,
                dtype="int64",
            )
        )
    return result

storage_op = store_blocks(darray, zarray)
da.compute(storage_op)

([array([0, 0, 0, 0])],)

In [17]:
da.from_array(zarray)

dask.array<array, shape=(16,), dtype=uint8, chunksize=(16,), chunktype=numpy.ndarray>

In [18]:
da.from_zarr('test.zarr', component='foo')

dask.array<from-zarr, shape=(16,), dtype=uint8, chunksize=(4,), chunktype=numpy.ndarray>

In [34]:
# Create a zarr array
import zarr
import numpy as np

data = np.arange(16, dtype='uint8').reshape(4,4) # 2D data

store_flat = zarr.DirectoryStore('test_flat.zarr')
zarray_flat = zarr.open(store_flat, # Root of the zarr hierarchy
                  path='foo',  # Path to the array I am creating
                  shape=data.shape,  # Array shape
                  dtype=data.dtype, # Array datatype
                  chunks=(2,2), # Array chunking
                  mode='w') # Storage access mode

zarray_flat[:] = 10

In [40]:
!tree -a test_flat.zarr

test_flat.zarr
├── .zgroup
└── foo
    ├── .zarray
    ├── 0.0
    ├── 0.1
    ├── 1.0
    └── 1.1

1 directory, 6 files


In [37]:
store_nested = zarr.NestedDirectoryStore('test_nested.zarr')
zarray_nested = zarr.open(store_nested, # Root of the zarr hierarchy
                  path='foo',  # Path to the array I am creating
                  shape=data.shape,  # Array shape
                  dtype=data.dtype, # Array datatype
                  chunks=(2,2), # Array chunking
                  mode='w') # Storage access mode

zarray_nested[:] = 10

In [39]:
!tree -a test_nested.zarr

test_nested.zarr
├── .zgroup
└── foo
    ├── .zarray
    ├── 0
    │   ├── 0
    │   └── 1
    └── 1
        ├── 0
        └── 1

3 directories, 6 files
